In [2]:
#combining all datasets using map function

import glob
import pandas as pd 

#map renewables

df_renew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_renewable/*.csv')))
df_renew.rename(columns={'Country or Area':'Geo'}, inplace=True)

#map nonrenewables

df_nonrenew = pd.concat(map(pd.read_csv, glob.glob('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/raw_data_nonrenewable/*.csv')))
df_nonrenew.rename(columns={'Country or Area':'Geo'}, inplace=True)

df = pd.concat([df_renew, df_nonrenew])

#split commodity and transaction into two columns
#defining column to split and which delimiter
ct = df['Commodity - Transaction'].str.split('-')

#assigning new columns
df['Commodity'] = ct.str.get(0)
df['Transaction'] = ct.str.get(1)

#lowercase and remove trailing and leading spaces
df['Commodity'] = df['Commodity'].str.lower()
df['Commodity'] = df['Commodity'].str.strip()

df['Transaction'] = df['Transaction'].str.lower()
df['Transaction'] = df['Transaction'].str.strip()

df = df[df['Transaction'] == 'production']


In [3]:
import pandas as pd 
unit_conv = pd.read_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_energy_units_conv_2.csv')

df_merge = pd.merge(df,  
                     unit_conv,  
                     on ='Commodity', 
                     how ='left') 

df_merge['TJ'] = df_merge['Quantity'] * df_merge['Conv_mt_to_tj']
df_merge.sample(3)

,Geo,Commodity - Transaction,Year,Unit,Quantity,Quantity Footnotes,Commodity,Transaction,Description,Units,Conv_mt_to_tj,r_type,TJ
72239,Qatar,Other oil products n.e.c. - Production,2005.0,"Metric tons, thousand",455.000000,NaN,other oil products n.e.c.,production,NaN,"Metric tons, thousand",40.20,nonrenewable,1.829100e+04
59916,United States,Motor Gasoline - Production,2014.0,"Metric tons, thousand",341919.000000,NaN,motor gasoline,production,NaN,"Metric tons, thousand",44.30,nonrenewable,1.514701e+07
15380,Tunisia,Fuelwood - Production,2016.0,"Cubic metres, thousand",4297.974822,NaN,fuelwood,production,Wood used as fuel,"Cubic meters, thousand",9.14,renewable,3.928349e+04


In [4]:

#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
lt_2018 = df_merge[df_merge['Year'] == 2018]
#greate geo group
tot_grp_country = lt_2018.groupby(['Geo'])
#sum TJ by geo group
df_tot_country = tot_grp_country['TJ'].agg(['sum'])

#sum all renewables
lt_2018_r = df_merge[(df_merge['Year'] == 2018) & (df_merge['r_type'] == 'renewable')]
tot_r_grp_country = lt_2018_r.groupby(['Geo'])
df_tot_country_r = tot_r_grp_country['TJ'].agg(['sum'])

#sum all nonrenewables
lt_2018_n = df_merge[(df_merge['Year'] == 2018) & (df_merge['r_type'] == 'nonrenewable')]
tot_n_grp_country = lt_2018_n.groupby(['Geo'])
df_tot_country_n = tot_n_grp_country['TJ'].agg(['sum'])

In [11]:
#merge total, renewable, and nonrenewable dataframes

countries_r = pd.merge(df_tot_country,  
                     df_tot_country_r,  
                     on ='Geo', 
                     how ='left') 

countries_r_n = pd.merge(countries_r,
                    df_tot_country_n,
                    on = 'Geo', 
                    how = 'left')

countries_r_n.rename(columns={'sum_x':'total', 'sum_y': 'renew', 'sum' : 'nonrenew'}, inplace=True)
countries_r_n['percent_renewable'] = (countries_r_n['renew'] / (countries_r_n['renew'] + countries_r_n['nonrenew'])) * 100
countries_r_n.sort_values(by='percent_renewable', ascending=False, inplace=True)
#allsources_r_n['diff'] = allsources_r_n['renew'] + allsources_r_n['nonrenew']#

countries_r_n.head()
# countries_r_n.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/gdp/perc_renewable_by_country.csv')

In [6]:
#to get the breakdown of types for the top 10 'natural' group

#sum all energy types, df_merge is the combination of unit conversion table and raw data to sum total TJ
#filter by year
for geo in df_merge['Geo'].unique() : 
    country_types = df_merge[(df_merge['Year'] == 2018) & (df_merge['Geo'] == geo) & (df_merge['r_type'] == 'renewable')]
    #greate geo group
    country_type_grp = country_types.groupby(['Commodity'])
    #sum TJ by geo group
    country_types = country_type_grp['TJ'].agg(['sum'])
    country_types.sort_values(by='sum', ascending=False, inplace=True)
    country_types.head(20)


    #comment out unless you want to export 200+ csv files:
    #country_types.to_csv('C:/Users/Rachel/OneDrive/Documents/Springboard/CAPSTONE 2/energy/renewable_types_by_country/' + geo + '.csv')

In [7]:
#export results to csv for tableau viz, keep commented out

#countries_r_n.to_csv('countries_percent_renewable_2013.csv')